In [1]:
from nnunet_ext.network_architecture.nca.NCA2D import NCA2D
import torch
import torch.nn.functional as F
torch.set_grad_enabled(False)

In [2]:
device = "cuda"
x = torch.randn(1,1,4,4, device=device)

In [3]:
hidden_size0 = 64
nca0 = NCA2D(num_channels=32, num_input_channels=1, num_classes=0, hidden_size=hidden_size0, fire_rate=1, num_steps=10, use_norm=False)
nca0.to(device)

hidden_size1 = 64
nca1 = NCA2D(num_channels=32, num_input_channels=0, num_classes=0, hidden_size=hidden_size1, fire_rate=1, num_steps=10, use_norm=False)
nca1.fc0 = torch.nn.Conv2d(2 * (nca0.num_channels + nca1.num_channels), hidden_size1, kernel_size=1)
nca1.to(device);

In [4]:
state = nca0.make_state(x)
out0 = nca0.forward_internal(state)

In [31]:
def extended_nca_update(base_nca, new_nca, states):
    delta_state0 = base_nca.conv(states[0])
    delta_state1 = new_nca.conv(states[1])
    delta_state1 = torch.cat([states[0], states[1], delta_state0, delta_state1], dim=1)
    delta_state0 = torch.cat([states[0], delta_state0], dim=1)
    delta_state0 = base_nca.fc0(delta_state0)
    delta_state1 = new_nca.fc0(delta_state1)
    print(delta_state1[:,-1])
    delta_state0 = base_nca.batch_norm(delta_state0)
    delta_state1 = new_nca.batch_norm(delta_state1)
    delta_state0 = F.relu(delta_state0, inplace=False)
    delta_state1 = F.relu(delta_state1, inplace=False)
    delta_state0 = base_nca.fc1(delta_state0)
    delta_state1 = new_nca.fc1(delta_state1)
    temp_state0 = states[0][:, base_nca.num_input_channels:] + delta_state0
    temp_state0 = torch.cat([states[0][:, :base_nca.num_input_channels], temp_state0], dim=1)
    return [temp_state0, states[1] + delta_state1]

In [37]:
def extended_nca_updated_merged(base_nca, new_nca, states):
    merged_nca = NCA2D(num_channels=base_nca.num_channels + new_nca.num_channels, num_input_channels=base_nca.num_input_channels, 
                       num_classes=16, hidden_size=hidden_size0 + hidden_size1, fire_rate=1, num_steps=10, use_norm=False).to(device)

    merged_nca.conv.weight = torch.nn.Parameter(torch.cat([base_nca.conv.weight, new_nca.conv.weight], dim=0))
    merged_nca.conv.bias = torch.nn.Parameter(torch.cat([base_nca.conv.bias, new_nca.conv.bias], dim=0))

    base_nca_fc0_upper = base_nca.fc0.weight[:, :base_nca.num_channels, :, :]
    base_nca_fc0_lower = base_nca.fc0.weight[:, base_nca.num_channels:, :, :]

    base_nca_fc0_w_zeros = torch.cat([base_nca_fc0_upper, torch.zeros(hidden_size0, new_nca.num_channels, 1, 1, device=device),
                                      base_nca_fc0_lower, torch.zeros(hidden_size0, new_nca.num_channels, 1, 1, device=device)], dim=1)
    fc0 = torch.cat([base_nca_fc0_w_zeros, new_nca.fc0.weight], dim=0)
    assert merged_nca.fc0.weight.shape == fc0.shape, f"{merged_nca.fc0.weight.shape}, {base_nca_fc0_w_zeros.shape}"
    merged_nca.fc0.weight = torch.nn.Parameter(fc0)
    merged_nca.fc0.bias = torch.nn.Parameter(torch.cat([base_nca.fc0.bias, new_nca.fc0.bias], dim=0))


    base_nca_fc1_w_zeros = torch.cat([base_nca.fc1.weight, torch.zeros(base_nca.num_channels-base_nca.num_input_channels, hidden_size1, 1, 1, device=device)], dim=1)
    new_nca_fc1_w_zeros = torch.cat([torch.zeros(new_nca.num_channels, hidden_size0, 1, 1, device=device), new_nca.fc1.weight], dim=1)
    fc1 = torch.cat([base_nca_fc1_w_zeros, new_nca_fc1_w_zeros], dim=0)
    assert merged_nca.fc1.weight.shape == fc1.shape, f"{merged_nca.fc1.weight.shape}, {fc1.shape}"
    merged_nca.fc1.weight = torch.nn.Parameter(fc1)
    assert base_nca.fc1.bias is None and new_nca.fc1.bias is None
    assert merged_nca.fc1.bias is None


    temp = merged_nca.conv(states)
    temp = torch.cat([states, temp], dim=1)
    temp = merged_nca.fc0(temp)
    print(merged_nca.fc0.weight[...,0,0])
    print(temp[:,-1])
    assert isinstance(base_nca.batch_norm, torch.nn.Identity)
    assert isinstance(new_nca.batch_norm, torch.nn.Identity)
    temp = F.relu(temp, inplace=False)
    temp = merged_nca.fc1(temp)

    temp = temp + states[:, base_nca.num_input_channels:]
    return temp

In [38]:
state1 = torch.zeros(x.shape[0], nca0.num_channels - nca0.num_input_channels,
                    x.shape[2], x.shape[3], device=x.device)
state2 = torch.zeros(x.shape[0], nca1.num_channels, x.shape[2], x.shape[3], device=x.device)
states = [torch.cat([x, state1], dim=1), state2]


states = torch.cat([states[0], states[1]], dim=1)

for _ in range(1):
    states = extended_nca_updated_merged(nca0, nca1, states)
    states = torch.cat([x, states], dim=1)
states_merged = states.clone()

tensor([[ 0.0155, -0.0035, -0.0424,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1175,  0.0107,  0.0003,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1122, -0.1183,  0.1130,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0432,  0.0065, -0.0722,  ...,  0.0515, -0.0363,  0.0568],
        [-0.0761,  0.0283,  0.0384,  ...,  0.0037,  0.0848, -0.0474],
        [ 0.0684, -0.0528,  0.0770,  ...,  0.0275, -0.0393, -0.0378]],
       device='cuda:0', requires_grad=True)
tensor([[[-0.1552, -0.1157,  0.0838, -0.0747],
         [-0.0929, -0.0449, -0.0648, -0.0343],
         [-0.0810, -0.0479, -0.1444, -0.0886],
         [-0.0298, -0.0282, -0.0342,  0.0534]]], device='cuda:0')


In [39]:
state1 = torch.zeros(x.shape[0], nca0.num_channels - nca0.num_input_channels,
                    x.shape[2], x.shape[3], device=x.device)
state2 = torch.zeros(x.shape[0], nca1.num_channels, x.shape[2], x.shape[3], device=x.device)
states = [torch.cat([x, state1], dim=1), state2]

for _ in range(1):
    states = extended_nca_update(nca0, nca1, states)

tensor([[[-0.1552, -0.1157,  0.0838, -0.0747],
         [-0.0929, -0.0449, -0.0648, -0.0343],
         [-0.0810, -0.0479, -0.1444, -0.0886],
         [-0.0298, -0.0282, -0.0342,  0.0534]]], device='cuda:0')


In [40]:
(states_merged - torch.cat(states, dim=1)).abs().max()

tensor(0., device='cuda:0')